<a href="https://colab.research.google.com/github/BillTanekeu/TP-TPE-INF-3236-Groupe-8/blob/main/Tp3_KNN_sur_les_donn%C3%A9es_spam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


#TP:  KNN sur les données "spam"


> **Groupe-8**


* Tanekeu Bill Jordan-------------------------------19M2459

* Ndongo Ndongo Papia Ange Borel--------19M2308

* Ghomsi Sikali Diane Laure---------------------18U2202

* Tchuimo Wamba Jordan------------------------19M2396 


# Données utilisé: SMS Spam Collection Dataset

Lien de téléchargement: 

https://www.kaggle.com/datasets/uciml/sms-spam-collection-dataset/download

In [1]:
import os
import string
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import numpy as np
import pandas as pd


#Importation des données
Veuillez selectionner le fichier spam.csv téléchargé plus haut

In [2]:
from google.colab import files
import io
uploader = files.upload()

Saving spam.csv to spam.csv


In [3]:
def load_data():
    data = pd.read_csv(io.BytesIO(uploader["spam.csv"]), encoding = "ISO-8859-1")
    data = data.drop(columns=['Unnamed: 2', 'Unnamed: 3','Unnamed: 4'])
    data.columns = ["class", "message"]

    return data
data = load_data()

fonction de chargement du jeu de données.
après chargement du fichier .csv , il supprime les trois dernières colonnes qui nous sont inutiles. Ensuite on renomme les deux premières en respectivement  class et message

##Entête jeu de données

In [13]:
data.head

<bound method NDFrame.head of      class                                            message
0      ham  Go until jurong point, crazy.. Available only ...
1      ham                      Ok lar... Joking wif u oni...
2     spam  Free entry in 2 a wkly comp to win FA Cup fina...
3      ham  U dun say so early hor... U c already then say...
4      ham  Nah I don't think he goes to usf, he lives aro...
...    ...                                                ...
5567  spam  This is the 2nd time we have tried 2 contact u...
5568   ham              Will Ì_ b going to esplanade fr home?
5569   ham  Pity, * was in mood for that. So...any other s...
5570   ham  The guy did some bitching but I acted like i'd...
5571   ham                         Rofl. Its true to its name

[5572 rows x 2 columns]>

## Nombre de spam dans le jeu de données

In [14]:
len(data[data['class'] == "spam"])


747

In [15]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

#Prétraitement des données

- Suppression des symboles ou pontuaction
- Mettre toutes les lettres en minuscules

In [16]:
def preprocess_data(data):
    print("Preprocessing data...")
    
    punc = string.punctuation           #Liste de ponctuation 
    sw = stopwords.words('english')     # liste des mots vides
    for i in range(len(data)):
         # Supprime la ponctuation et les symboles courants 
        for item in punc:
            data[i] = data[i].replace(item, "")
         # Mettez toutes les lettres en minuscules et supprimez les mots vides
        splittedWords = data[i].split()
        newText = ""
        for word in splittedWords:
            if word not in sw:
                word = word.lower()
                newText = newText + " " + word  
        data[i] = newText
    return data


## resultat du pre-traitement

In [17]:

df1 = data
data_pre = preprocess_data(df1['message'])
data_pre

Preprocessing data...


0        go jurong point crazy available bugis n great...
1                                 ok lar joking wif u oni
2        free entry 2 wkly comp win fa cup final tkts ...
3                     u dun say early hor u c already say
4           nah i dont think goes usf lives around though
                              ...                        
5567     this 2nd time tried 2 contact u u å£750 pound...
5568                     will ì b going esplanade fr home
5569                          pity mood soany suggestions
5570     the guy bitching i acted like id interested b...
5571                                   rofl its true name
Name: message, Length: 5572, dtype: object

#Séparation données d'entraînement et test
On utilise train_test_split du module sklearn

27% des données est utilisé pour le test

In [20]:
def split_data(data):
    labels = data["class"]
    features =  data['message']
    features = preprocess_data(features)
    
    features = features.to_numpy()
    labels = labels.to_numpy()

    training_data, test_data, training_labels, test_labels = train_test_split(features, labels, test_size = 0.27, random_state = 42)  # division du data set 
    return training_data , test_data , training_labels , test_labels

training_data , test_data , training_labels , test_labels = split_data(data)

print("taille données d'entraînement : ", training_data.shape)
print("taille données de test : ", test_data.shape)


Preprocessing data...
taille données d'entraînement :  (4067,)
taille données de test :  (1505,)


Tout d'abord, il est nécessaire d'avoir les textes des e-mails dans un tableau numpy et les étiquettes dans un autre tableau.

La méthode train_test_split a ensuite été utilisée pour diviser les données en training_data, test_data, training_labels et test_labels. L'état aléatoire a été défini sur 42 pour garantir que la même sortie de mélange aléatoire sera obtenue à des fins de test. Après le fractionnement, training_data, test_data, training_labels et test_labels sont renvoyés.


In [24]:
comp = 0
for i in training_labels:
  if i=='spam':
    comp +=1
print(comp)

544


#L'algorithme KNN


In [25]:
def get_count(text):
    wordCounts = dict()
    for word in text.split():
        if word in wordCounts:
            wordCounts[word] += 1
        else:
            wordCounts[word] = 1
    
    return wordCounts



Cette fonction prend un seul texte d'e-mail et le divise à l'aide de split(). La fréquence d'occurrence de chaque mot dans l'e-mail est comptée et enregistrée dans wordCounts, qui est de type dictionnaire. Le dictionnaire wordCounts est ensuite renvoyé.

##  resultat de get_count() sur une données

In [26]:

get_count(training_data[1])

{'2': 1,
 '25': 1,
 'alone': 1,
 'buffy': 1,
 'c': 1,
 'feeling': 1,
 'freemsg': 1,
 'help08700621170150p': 1,
 'hey': 1,
 'home': 1,
 'im': 1,
 'love': 1,
 'men': 1,
 'msg': 1,
 'pix': 1,
 'qlynnbv': 1,
 'randy': 1,
 'reply': 1,
 'satisfy': 1,
 'send': 1,
 'stop': 2,
 'txts': 1}

#fonction euclidean_difference()
Cette fonction prend en compte un dictionnaire de nombre de mots d'un e-mail de test test_WordCounts et un autre dictionnaire d'un e-mail de formation, training_wordCounts. total stocke la somme de la différence au carré de la fréquence d'un mot dans l'e-mail de test et d'entraînement.

In [27]:
def euclidean_difference(test_WordCounts, training_WordCounts):
    total = 0
    for word in test_WordCounts:
        
        # si le mot est dans les deux e-mails, calculez la différence de comptage, mettez-la au carré et ajoutez-la au total
        if word in test_WordCounts and word in training_WordCounts:
            total += (test_WordCounts[word] - training_WordCounts[word])**2
            
            # pour supprimer les mots courants, pour accélérer le traitement dans la prochaine boucle for
            del training_WordCounts[word]
        
        # si mot dans l'e-mail de test uniquement, mettez le compte au carré et ajoutez au total
        else:
            total += test_WordCounts[word]**2
    
    # Mettez au carré le nombre de mots uniquement dans l'e-mail de formation et ajoutez-le au total
    for word in training_WordCounts:
        total += training_WordCounts[word]**2
     
    return total**0.5

## calcul de la distance entre la première données d'entraînement et la première données de test


In [28]:
euclidean_difference(get_count(training_data[1]), get_count(test_data[1]))

6.708203932499369

#fonction get_class()
Cette fonction prend en compte la liste des K plus proches voisins sélectionnés pour déterminer la classe de l'email de test en cours. spam_count et ham_count stockent la fréquence d'occurrence de chaque étiquette "spam" et étiquette "ham" respectivement dans les K voisins les plus proches sélectionnés.

si spam_count est supérieur à ham_count, cela signifie que l'e-mail de test actuel a plus tendance à être du spam, donc une chaîne "spam" est renvoyée comme étiquette prédite. Sinon, la chaîne "ham" est renvoyée comme étiquette prédite.

In [29]:
def get_class(selected_Kvalues): 
    spam_count = 0 
    ham_count = 0
    for value in selected_Kvalues:
      if value[0] == "spam":
        spam_count+= 1
      else:
        ham_count+= 1
    
    if spam_count > ham_count :
      return "spam"
    else:
      return "ham"

    

#fonction knn_classifier()

Le classifieur KNN ci dessous traite un jeu de données entier de facon à prendre chaque donnée contenu dans l'ensemble de test et calculer sa distance par rapport à toutes les données de l'ensemble de "train". 

Il stocke les resultats dans un tableau(similarity) puis ce tableau est trié dans l'ordre croissant des distances; en fonctioin du k défini , on consevera les k premières distances qui nous permettront d'évaluer la classe de la donnée de test correspondante. Chaque résultat est ainsi sauvegardé dans dans un tableau qui est renvoyé à la fin de la procédure.

In [30]:
# le module tqdm permet d'avoir une barre de progression
from tqdm import tqdm

In [32]:
def knn_classifier(training_data, training_labels, test_data, k ):
  print("Running KNN classifier...")
  result = []
  training_WordCounts = []

  for training_text in training_data:
      training_WordCounts.append(get_count(training_text))
  
  for test_text in tqdm(test_data):
      similarity = []
      test_WordCounts = get_count(test_text)
      
      for index in range(len(training_data)):
        euclidean_diff =  euclidean_difference(test_WordCounts , training_WordCounts[index])
        similarity.append([training_labels[index], euclidean_diff])
      
      similarity = sorted(similarity , key = lambda i:i[1])
      selected_Kvalues = []
      
      for i in range(k) : 
        selected_Kvalues.append(similarity[i])
      
      result.append(get_class(selected_Kvalues))
  
  return result


In [33]:
# execution de knn sur une données
knn_classifier(training_data, training_labels, test_data[1:2], 5 )

Running KNN classifier...


100%|██████████| 1/1 [00:00<00:00,  7.40it/s]


['ham']

#Exécution sur le jeu de test

In [37]:
k = 3    # ici on fixe par exemple le nombre de voisins à 11
result = knn_classifier(training_data, training_labels, test_data, k )

Running KNN classifier...


100%|██████████| 1505/1505 [00:38<00:00, 39.50it/s]


# accuracy-score

In [38]:
tsize = len(test_data)
accuracy = accuracy_score(test_labels[:tsize], result)

In [39]:
print("training data size\t:" + str(len(training_data)))
print("k value\t\t\t:" + str(k))
print("Samples tested\t\t:" + str(tsize))
print("% accuracy\t\t:" + str(accuracy*100))
print("number correct\t\t:" + str(int(accuracy*tsize)))
print("Number wrong\t\t:" + str(int((1-accuracy)*tsize)))

training data size	:4067
k value			:3
Samples tested		:1505
% accuracy		:89.03654485049833
number correct		:1340
Number wrong		:165
